In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 28 20:19:14 2018

@author: loewi
"""
import os
from time import time
from feature_extraction import __get

path = r'/Users/loewi/Documents/Pre_Learn/classification/20news-bydate/'
os.chdir(path)
#print(os.getcwd())

t0 = time() 

newsgroups_train = __get('20news-bydate-train')
newsgroups_test = __get('20news-bydate-test')

duration = time() - t0
print('%0.2fs get data package ：）'%duration)

X_train, X_test = newsgroups_train['data'], newsgroups_test['data']
y_train, y_test = newsgroups_train['docs'], newsgroups_test['docs']
print('X & y prepared')



2.84s get data package ：）
X & y prepared


In [3]:


from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score,StratifiedKFold,GridSearchCV
from pprint import pprint



In [4]:

# 调参
nbc = Pipeline([
                ('vect', CountVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b'
                        )),
                ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB()),
                ])
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 20000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
#    'tfidf__use_idf': (True, False),
#    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.1, 0.01, 0.001, 0.0001),
}    
cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = GridSearchCV(nbc,parameters,n_jobs=-1, verbose=1, cv = cv)
print("Performing grid search...")
print("pipeline:", [name for name, _ in nbc.steps])
print("parameters:")
pprint(parameters)
t0 = time()
clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % clf.best_score_)
print("Best parameters set:")
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))        
    


Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__alpha': (0.1, 0.01, 0.001, 0.0001),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__max_features': (None, 5000, 10000, 20000),
 'vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   41.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  9.6min finished


done in 583.502s

Best score: 0.842
Best parameters set:
	clf__alpha: 0.01
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (1, 2)


In [5]:
#%% predict
    
from sklearn import metrics  
clf_validated = Pipeline([
                ('vect', CountVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b',
                        max_df = 0.5,
                        ngram_range = (1, 2),
                        )),
                ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB(alpha = 0.01)),
                ])    
                
clf_validated.fit(X_train, y_train)  

cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
scores = cross_val_score(clf_validated, X_train, y_train,  cv = cv, scoring='accuracy')
print (scores)
score = scores.mean()
print("training accuracy: %0.3f (+/- %0.3f)" % (score, scores.std() * 2))
              
pred = clf_validated.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("testing accuracy:   %0.3f" % score)


[0.83605112 0.82877317 0.84673145 0.84823009 0.84789357]
training accuracy: 0.842 (+/- 0.016)
testing accuracy:   0.757
